In [1]:
# !pip install pysqlite3
# !pip install tqdm

In [2]:
import numpy as np
import pandas as pd
from numpy import random
import sqlite3

import re

import random

import warnings
warnings.filterwarnings('ignore')


cnxn = sqlite3.connect("reccar_0130.db")
cursor = cnxn.cursor() 

seed = 1998

In [3]:
area_list = ['Dobong', 'Dongdaemun', 'Dongjag', 'eunpyeong', 'geumcheon',
'Guro', 'GwanUck', 'Jongno', 'Junggu', 'Jungnang', 'Kongbuk',
'Kongdong', 'Kongjin', 'Kongnam', 'Kongseo', 'Mapo', 'Nowon', 'Seocho',
'Seodaemun', 'Seongbuk', 'Seongdong', 'songpa', 'Yangcheon',
'Yeongdeungpo', 'Yongsan']

len(area_list)

25

In [4]:
"""
data, rest 데이터 불러서 Concat 하기
"""
data = pd.DataFrame()
rest = pd.DataFrame()

def solve(x):
    a = re.findall(r'\d{4}년 \d{1,2}월 \d{1,2}일', str(x))
    try: return a[0]
    except: return np.NAN

cnt = 0
for area in area_list:
    tem = pd.read_csv(f'./user_csv/{area}.csv')
    tem['userid'] = tem['link'].apply(lambda x: str(re.findall(r'my/[a-z0-9]+/', x))[5:-3])
    tem['date'] = tem['user'].apply(lambda x: solve(x))

    tem = tem[['userid', 'rest', 'date']]
    tem = tem.drop_duplicates()
    data = pd.concat([data, tem])

    try: tem = pd.read_csv(f'./rest_csv/rest_concat_{area}.csv')
    except: continue
    tem = tem.drop_duplicates()
    #print(area)
    cnt += 1
    rest = pd.concat([rest, tem])   

data = data.drop_duplicates(['userid','rest']) # 30만개정도 빠짐
data = data[~data['rest'].isnull()] # 7개정도 null 값이 있더라
data = data[~(data['userid'] == '')]
data['date'] = pd.to_datetime(data['date'], format='%Y년 %m월 %d일')
data = data.sort_values(['userid', 'date'], ascending= [True, False])

data.reset_index(drop=True, inplace = True)
rest.reset_index(drop=True, inplace = True)
print(cnt)

25


In [5]:
"""
rest 값 전처리
"""
# 이미지 값 없는 것 tem 처리.
rest['image'] = rest['image'].fillna('tem') 

# 이미 사라진 식당으로 추정되는 값 위치 추척 안되도록 -1 넣음
rest['x'][rest['x'] == 'start'] = '-1'
rest['y'][rest['y'] == 'items'] = '-1'

rest['x'] = rest['x'].astype('int')
rest['y'] = rest['y'].astype('int')

# 음식점 태그 넣기 위한 사전 전처리
def make_tag(x):
    tem = x.split(',')
    return tem

def cnt_tag(x):
    return len(x)

rest['tag'] = rest['tag'].apply(lambda x : make_tag(x))
rest['tag_len'] = rest['tag'].apply(lambda x : cnt_tag(x))
rest['tag_len'].max() # 3

3

In [7]:
"""
tag_tag 파일 이용해서 태그 만들기
"""

tag = pd.read_csv('data/tag_tag.csv')
tag_dic = tag.set_index('소분류').T.to_dict()

def make_tag_1(x):
    return x['tag'][0]

def make_tag_2(x):
    if x['tag_len'] >= 2:
        return x['tag'][1]
    else:
        return x['tag'][0]

def make_tag_3(x):
    if x['tag_len'] >= 3:
        return x['tag'][2]
    else:
        return x['tag'][0]


rest['tag_1'] = rest.apply(lambda x : make_tag_1(x), axis = 1)
rest['tag_2'] = rest.apply(lambda x : make_tag_2(x), axis = 1)
rest['tag_3'] = rest.apply(lambda x : make_tag_3(x), axis = 1)

rest['tag_1'] = rest['tag_1'].apply(lambda x : tag_dic[x]['대분류'])
rest['tag_2'] = rest['tag_2'].apply(lambda x : tag_dic[x]['대분류'])
rest['tag_3'] = rest['tag_3'].apply(lambda x : tag_dic[x]['대분류'])

In [8]:
normal_img = {'중식': 'https://m.eco-fresh.co.kr/web/product/big/202207/d50ada11b2faf8572fc42753d3aeba37.jpg',
'한식' : 'https://www.fsnews.co.kr/news/photo/201902/32707_27510_4314.jpg',
'음식' : 'https://www.fsnews.co.kr/news/photo/201902/32707_27510_4314.jpg',
'음식아님' : 'https://www.fsnews.co.kr/news/photo/201902/32707_27510_4314.jpg',
'일식' : 'https://imgnn.seoul.co.kr/img/upload/2016/03/24/SSI_20160324104448_V.jpg',
'양식' : 'https://pds.joongang.co.kr/news/component/htmlphot…01901/23/ae150d3b-957a-461c-bf88-fff1ff6a648f.jpg',
'동남아음식' : 'https://mp-seoul-image-production-s3.mangoplate.com/mango_pick/1jflm8lkadguvz.jpg',
'분식&샐러드' : 'http://cdn.ksilbo.co.kr/news/photo/202008/769877_439211_1542.jpg',
'해산물' : 'https://image.homeplus.kr/td/5e9204f0-df66-49ef-82be-160a5f46f67a',
'고기' : 'https://post-phinf.pstatic.net/MjAyMDAzMDNfMTcg/MDAxNTgzMTkwNjA3ODQ5.kUXPHqGJ2xPDSu_3FiEoFC3kY9QyQ_g9CziCGrFSDuEg.LpCfOTbc5qth9d-GKzGv9jwj2VKhcqmPHp5cp1KJYEsg.JPEG/IM_food02.jpg?type=w1200',
'카페&디저트' : 'https://cdn.kormedi.com/wp-content/uploads/2021/05/0002313329_001_202105200942055171-580x387.jpg',
'술집' : 'https://img.khan.co.kr/news/2021/07/14/2021071501001702300164451.jpg',
'패스트푸드' : 'https://img.hankyung.com/photo/201908/01.20330262.1.jpg'}

In [9]:
"""
tag 예외처리
"""
rest['tag_1'][rest['restaurant'] == '부산어묵'] = '분식'
rest['tag_2'][rest['restaurant'] == '부산어묵'] = '분식'
rest['tag_3'][rest['restaurant'] == '부산어묵'] = '분식'

rest['tag_1'][rest['restaurant'] == '하모네꼬'] = '디저트'
rest['tag_2'][rest['restaurant'] == '하모네꼬'] = '디저트'
rest['tag_3'][rest['restaurant'] == '하모네꼬'] = '디저트'

rest['tag_1'][rest['restaurant'] == '람보르끼니'] = '양식'
rest['tag_2'][rest['restaurant'] == '람보르끼니'] = '양식'
rest['tag_3'][rest['restaurant'] == '람보르끼니'] = '양식'

rest['tag_1'][rest['restaurant'] == '쟌스키친'] = '디저트'
rest['tag_2'][rest['restaurant'] == '쟌스키친'] = '디저트'
rest['tag_3'][rest['restaurant'] == '쟌스키친'] = '디저트'

rest['tag_1'][(rest['tag_1'] == '카페') | (rest['tag_1'] == '디저트')] = '카페&디저트'
rest['tag_2'][(rest['tag_2'] == '카페') | (rest['tag_2'] == '디저트')] = '카페&디저트'
rest['tag_3'][(rest['tag_3'] == '카페') | (rest['tag_3'] == '디저트')] = '카페&디저트'

rest['tag_1'][(rest['tag_1'] == '분식') | (rest['tag_1'] == '샐러드')] = '분식&샐러드'
rest['tag_2'][(rest['tag_2'] == '분식') | (rest['tag_2'] == '샐러드')] = '분식&샐러드'
rest['tag_3'][(rest['tag_3'] == '분식') | (rest['tag_3'] == '샐러드')] = '분식&샐러드'

rest['image'] = rest.apply(lambda x : x['image'] if x['image'] != 'tem' else normal_img[x['tag_1']], axis = 1)

In [10]:
"""
cold 유저 제외하고(6개이상 기록 있는 유저만) 인코딩 진행
(1692677 명 중 382940명만 유효유저)
"""
# Cold 유저 제외하기
_user = data['userid'].value_counts().reset_index()
_user.columns = ['userid', 'cnt']
data = pd.merge(data, _user, how = 'left', on = 'userid')
data = data[data['cnt'] > 5].reset_index(drop = True)


# 유저, 아이템 인코딩 진행
user2idx = {user:idx for idx, user in enumerate(data['userid'].unique())}
idx2user = {idx:user for idx, user in enumerate(data['userid'].unique())}

## 패딩 때문에 0 비워놓음
item2idx = {item:(idx+1) for idx, item in enumerate(data['rest'].unique())}
idx2item = {(idx+1):item for idx, item in enumerate(data['rest'].unique())}

data['user_code'] = data['userid'].map(user2idx)
data['rest_code'] = data['rest'].map(item2idx)
rest['rest_code'] = rest['url'].map(item2idx)

In [11]:
"""
Train/Test 데이터 분할
"""
data['point'] = data['cnt'] // 10 + 2
data['tem'] = 1

# 시계열을 고려한 Test_set

data = data.sort_values(['user_code', 'date'], ascending= [True, False])
data['seq'] = data.groupby('user_code')['tem'].apply(lambda x : x.cumsum())
train_time = data[data['seq'] > data['point']]
test_time = data[data['seq'] <= data['point']]

# 랜덤하게 한 값을 뚫은 Test_set
random.seed(seed)
data['rand'] = data['rest'].apply(lambda x : random.random())
data = data.sort_values(['user_code', 'rand'], ascending= True)
data['seq'] = data.groupby('user_code')['tem'].apply(lambda x : x.cumsum())

train_rand = data[data['seq'] > data['point']]
test_rand = data[data['seq'] <= data['point']]

In [13]:
# 시계열 모델을 위해 모든 train 데이터 시간순으로 정렬
train_rand = train_rand.sort_values(['user_code', 'date'], ascending= True)
train_time = train_time.sort_values(['user_code', 'date'], ascending= True)
data = data.sort_values(['user_code', 'date'], ascending= True)

In [14]:
"""
Data 기준 시간 순서대로 user 기록 저장(시계열 모델을 위해)
"""
user = data.groupby('user_code')['rest_code'].unique().to_frame().reset_index()
user['user'] = user['user_code'].map(idx2user)
user['rest_code'] = user['rest_code'].apply(lambda x : list(x))
user['rest_code'] = user['rest_code'].astype('str')
user.head()

,user_code,rest_code,user
0,0,"[20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9...",5b61c7658f8242cb2a1b1028
1,1,"[76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 6...",5b62c91cd898db2e71efcc59
2,2,"[88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77]",5b62e55e10599e4793d54f8a
3,3,"[98, 97, 96, 95, 94, 93, 92, 91, 90, 89]",5b62e6c46daf19a0c8d33997
4,4,"[147, 146, 145, 144, 143, 142, 141, 140, 139, ...",5b62e6e8c5d392d7522d1436


In [15]:
"""
rest 개수 제작(인기도 기반 추천 위해)
"""
def raking(x):
    try:
        cnt = rest_rank[x]
        return cnt
    except:
        return -1

rest_rank = data['rest_code'].value_counts()
rest['cnt'] = rest['rest_code'].apply(lambda x : raking(x))
rest = rest[~rest['rest_code'].isnull()]
rest['rest_code'] = rest['rest_code'].astype('int')
rest = rest[['restaurant','tag_1','url','x','y','image','rest_code', 'cnt']]

In [16]:
"""
데이터 csv 저장 (model를 위한 data + data_all을 구축하기 위한 data 2개 저장)
"""
paths = ['../model/data/', 'data/']

for path in paths:
    data[['userid', 'rest', 'user_code', 'rest_code', 'date']].to_csv(path+'data.csv', index = False)

    train_rand[['userid', 'rest', 'user_code', 'rest_code', 'date']].to_csv(path+'train_rand.csv', index = False)
    test_rand[['userid', 'rest', 'user_code', 'rest_code']].to_csv(path+'test_rand.csv', index = False)

    train_time[['userid', 'rest', 'user_code', 'rest_code', 'date']].to_csv(path+'train_time.csv', index = False)
    test_time[['userid', 'rest', 'user_code', 'rest_code']].to_csv(path+'test_time.csv', index = False)

    user.to_csv(path+'user.csv', index = False)
    rest.to_csv(path+'rest.csv', index = False)

In [19]:
"""
rest 테이블 DB 저장
"""
# cursor.execute("DROP TABLE rest")
# cnxn.commit()  # this commits changes to the database

cursor.execute("CREATE TABLE rest ("
               "name VARCHAR(255),"
               "tag VARCHAR(511),"
               "url INT(127),"
               "x INT(127),"
               "y INT(127),"
               "image VARCHAR(511),"
               "rest_code INT(127),"
               "cnt INT(127))"
)
cnxn.commit()  # this commits changes to the database

a = rest.values.tolist()
for i in range(len(a)):
    a[i] = tuple(list(a[i]))

for i in a:
    #print(i)
    cursor.execute("INSERT INTO rest VALUES (?, ?, ?, ?, ?, ?, ?,?)", i)
    #except: pass
cnxn.commit()

In [21]:
"""
user 방문 기록(rest_code) 테이블 DB 저장
"""
# cursor.execute("DROP TABLE user")
# cnxn.commit()  # this commits changes to the database

cursor.execute("CREATE TABLE user ("
               "user_code INT(127),"
               "rest_code VARCHAR(16383),"
               "user VARCHAR(255))"
)
cnxn.commit()  # this commits changes to the database

a = user.values.tolist()
for i in range(len(a)):
    a[i] = tuple(list(a[i]))

for i in a:
    #print(i)
    cursor.execute("INSERT INTO user VALUES (?, ?, ?)", i)
    #except: pass
cnxn.commit()

In [23]:
"""
positive, negative 기록 저장
"""
# cursor.execute("DROP TABLE positive")
# cnxn.commit()  # this commits changes to the database
# cursor.execute("DROP TABLE negative")
# cnxn.commit()  # this commits changes to the database

cursor.execute("CREATE TABLE positive ("
               "user VARCHAR(255),"
               "url INT(127),"
               "model VARCHAR(255))"
)
cnxn.commit()  # this commits changes to the database

cursor.execute("CREATE TABLE negative ("
               "user VARCHAR(255),"
               "url INT(127),"
               "model VARCHAR(255))"
)
cnxn.commit()  # this commits changes to the database